# Tunga Shopee

*Pesquisar no shopee os seguintes produtos*

- flipper zero
- Tv box 
- bloqueador de sinal
- carregador de bateria de celular
- jammer
- reforçador de celular
- bateria de celular
- transmissor fm

A pesquisa também deve registrar a quantidade de produtos localizados em cada plataforma com a respectiva  palvra-chave

In [81]:
produtos_para_pesquisa = [
#    'flipper zero',
#     'tvbox',
#     'bloqueador de sinal',
#     'carregador de bateria de celular',
#     'jammer bloqueador',
#     'reforçador de celular',
#     'bateria de celular',
     'transmissor fm']#
# ]

### importando bibliotecas

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd

from time import sleep
from datetime import datetime
import re
from urllib.parse import quote

### configurações do web browser

In [3]:
import requests
from urllib.parse import quote
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
#funções do webdriver
def get_driver() -> webdriver:
    """
    Retorna um objeto WebDriver para o Chrome.
    """

    # create object for chrome options

    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox') 
    chrome_options.add_argument('disable-notifications') 
    chrome_options.add_argument('--disable-infobars') 
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-popup-blocking")
    chrome_options.add_argument("--disable-translate")
    chrome_options.add_argument("--disable-web-security")
    chrome_options.add_argument("--disable-logging")
    chrome_options.add_argument("--disable-plugins-discovery")

    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    chrome_options.add_argument(f"user-agent={user_agent}")


    driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()),
                                  options = chrome_options)  
    
    driver.maximize_window()# Maximiza a janela
    driver.implicitly_wait(5)
    
    return driver


In [5]:
driver = get_driver() 
actions = ActionChains(driver)
driver.get('https://www.shopee.com.br')

In [ ]:
# # navegar até a shopee
# driver = get_driver() 
# driver.get('https://www.shopee.com.br/')
# sleep(3)
# elements = driver.find_elements(By.TAG_NAME, 'button')

# # clica em aceitar todos os cookies
# for e in elements:
#     if e.text == 'Aceitar todos os cookies':
#         actions.move_to_element(e)
#         actions.perform()
#         e.click()
#         break
        
# sleep(3)

# # clica em algum lugar da página
# element = driver.find_element(By.ID, 'main')
# actions.move_to_element(element).click().perform()
# sleep(3)

# # clicar para entrar
# e = driver.find_elements(By.PARTIAL_LINK_TEXT, 'Entre')[0]
# actions.move_to_element(e).click().perform()

# # fazer login
# telefone = ''
# senha    = ''

# campo_telefone = driver.find_elements(By.CLASS_NAME, 'pDzPRp')[0]
# campo_senha = driver.find_elements(By.CLASS_NAME, 'pDzPRp')[1]

# actions.move_to_element(campo_telefone).click().perform()
# for i in telefone:
#     campo_telefone.send_keys(i)
#     sleep(0.5)

# actions.move_to_element(campo_senha).click().perform()
# for i in senha:
#     campo_senha.send_keys(i)
#     sleep(0.5)

# # clicar para entrar
# elements = driver.find_elements(By.TAG_NAME, 'button')
# for e in elements:
#     if e.text == 'ENTRE':
#         actions.move_to_element(e).click().perform()
#         break
        
# sleep(3)

# # clica em algum lugar da página
# e = driver.find_element(By.CSS_SELECTOR, 'body')
# actions.move_to_element(e).click().perform()
# sleep(3)
# search_box = driver.find_element(By.CLASS_NAME, 'shopee-searchbar-input__input')
# actions.move_to_element(search_box).click().perform()
# sleep(0.5)

### capturar as quantidades de cada produto

In [ ]:
## percorrer páginas
try:
    ultima_pagina = int(driver.find_element(By.XPATH,'//*[@id="main"]/div/div[2]/div/div/div[2]/div[3]/div[1]/div[2]/div/span[2]').text)
except Exception as e:
    print('não achou 1')
    try:
        # <span class="shopee-mini-page-controller__total">6</span>
        ultima_pagina = int(driver.find_elements(By.CLASS_NAME,'shopee-mini-page-controller__total')[0].text)
    except Exception as e:
        print('não achou 2')
        ultima_pagina = 1
        
ultima_pagina

In [ ]:
quantidade_produtos = []
for i in produtos_para_pesquisa:
    query = quote(i)
    url = f'https://shopee.com.br/search?keyword={query}'
    driver.get(url)
    sleep(3)

    ## percorrer páginas
    try:
        ultima_pagina = int(driver.find_element(By.XPATH,'//*[@id="main"]/div/div[2]/div/div/div[2]/div[3]/div[1]/div[2]/div/span[2]').text)
    except Exception as e:
        pass #print('não achou 1')
        try:
            # <span class="shopee-mini-page-controller__total">6</span>
            ultima_pagina = int(driver.find_elements(By.CLASS_NAME,'shopee-mini-page-controller__total')[0].text)
        except Exception as e:
            # print('não achou 2')
            ultima_pagina = 1

        quantidade = ultima_pagina * 60
    
    
    # Obter a data e hora atuais
    data_hora_atual = datetime.now()
    data_hora_atual = f'{data_hora_atual.day}/{data_hora_atual.month}/{data_hora_atual.year} {data_hora_atual.hour}:{data_hora_atual.minute}:{data_hora_atual.second}'
    
    quantidade_produtos.append({'plataforma' :'Shopee',
                                'produto'    : i,
                                'quantidade' : quantidade,
                                'data_coleta': data_hora_atual
                               })

#quantidade_produtos

### salvar resultados

In [ ]:
df = pd.DataFrame(quantidade_produtos)
df.to_csv('resultados_quantidades_shopee.csv', index = False, encoding = 'utf-8',  sep =';')
df

### capturar os links

In [82]:
set_produtos = set()

for i in produtos_para_pesquisa:
    query = quote(i)
    url = f'https://shopee.com.br/search?keyword={query}'
    driver.get(url)
    sleep(5)

    ## percorrer páginas
    try:
        ultima_pagina = int(driver.find_element(By.XPATH,'//*[@id="main"]/div/div[2]/div/div/div[2]/div[3]/div[1]/div[2]/div/span[2]').text)
    except Exception as e:
        pass # print(e)
        try:
            # <span class="shopee-mini-page-controller__total">6</span>
            ultima_pagina = int(driver.find_elements(By.CLASS_NAME,'shopee-mini-page-controller__total')[0].text)
        except Exception as e:
            print(e)
            ultima_pagina = 1

    for i in range(ultima_pagina):

        print('pesquisando pág', i+1, 'de', ultima_pagina)

        # rolagem para esperar carregar os produtos
        for i in range(20):
            try:
                driver.execute_script("window.scrollBy(0, 100)")
                sleep(0.3)
            except:
                break

        div_links = driver.find_element(By.CSS_SELECTOR, '#main > div > div.dYFPlI > div > div > div.sdzgsX > div.shopee-search-item-result > div.row.shopee-search-item-result__items')
        a_elements = div_links.find_elements(By.TAG_NAME, 'a')

        for a in a_elements:
            set_produtos.add(a.get_attribute('href'))

        button = driver.find_elements(By.CLASS_NAME, 'shopee-icon-button--right')[0]
        actions.move_to_element(button).click().perform()
        sleep(3)



pesquisando pág 1 de 17
pesquisando pág 2 de 17
pesquisando pág 3 de 17
pesquisando pág 4 de 17
pesquisando pág 5 de 17
pesquisando pág 6 de 17
pesquisando pág 7 de 17
pesquisando pág 8 de 17
pesquisando pág 9 de 17
pesquisando pág 10 de 17
pesquisando pág 11 de 17
pesquisando pág 12 de 17
pesquisando pág 13 de 17
pesquisando pág 14 de 17
pesquisando pág 15 de 17
pesquisando pág 16 de 17
pesquisando pág 17 de 17


In [83]:
print(len(set_produtos))

395


In [84]:
links_produtos = list(set_produtos)
links_produtos[0]

'https://shopee.com.br/Carregador-turbo-adaptador-veicular-de-isqueiro-de-carro-c-2-sa%C3%ADdas-USB-(altomex-aluminio)-i.313196924.20057591504?sp_atk=c3d79015-b201-4bbd-ab5b-fd9097577780&xptdk=c3d79015-b201-4bbd-ab5b-fd9097577780'

### navegar pelos produtos

In [85]:
falhas = []
produtos = []
final = len(links_produtos)

for link in links_produtos:

    final = final -1
    if final % 10 == 0:
        print(final)

    try:
        driver.get(link)
        sleep(3)

        for i in range(10):
            try:
                driver.execute_script("window.scrollBy(0, 100)")
                time.sleep(0.5)
            except:
                break

        try:
            titulo_produto = driver.find_element(By.XPATH,
                                                 '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[1]'
                                                ).text
        except:
            titulo_produto = '-'

        try:
            id_vendedor    = driver.find_element(By.XPATH,
                                                 '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[1]/div[1]/div/div[1]'
                                                ).text
        except:
            id_vendedor    = '-'

        try:      
            qtde_vendida = driver.find_element(By.XPATH,
                                               '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[2]/div[2]/div[1]'
                                              ).text
        except:
            qtde_vendida = '0'

        try:
            localizacao = driver.find_element(By.XPATH,
                                                '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[4]/div/div[2]/div/div[2]/div[2]/div[1]/div[2]/div[1]'
                                             ).text
        except:
            localizacao = ''

        try:
            preco = driver.find_element(By.XPATH,
                                       '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[3]/div/div/div/div/div[1]'
                                       ).text

            if preco.find('-'):
                preco = preco.split('-')
                preco_minimo = preco[0][2:]
                preco_maximo = preco[-1][3:]
            else:
                preco_minimo = preco[2:]
                preco_maximo = '-'
        except:
            preco_minimo = '-'
            preco_maximo = '-'

        try:
            preco_promo = driver.find_element(By.XPATH,
                                       '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[3]/div/div/div/div/div[2]/div[1]'
                                       ).text

            if preco_promo.find('-'):
                preco_promo = preco_promo.split('-')
                preco_promo_minimo = preco_promo[0][2:]
                preco_promo_maximo = preco_promo[-1][3:]
            else:
                preco_promo_minimo = preco_promo[2:]
                preco_promo_maximo = '-'

        except:
                preco_promo_minimo = '-'
                preco_promo_maximo = '-'


        try:
            detalhes_produto = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[2]/div[1]/div[1]/div[1]/div[2]'
                                                  ).get_attribute('innerHTML')
            detalhes_produto = detalhes_produto.replace('</a>','|</a>')

            lista_chaves = re.findall(r'<label[^>]*>(.*?)</label>', detalhes_produto)

            # criar um objeto Beautiful Soup a partir do HTML do div
            soup = BeautifulSoup(detalhes_produto, 'html.parser')

            # remover todas as tags <label> e svg
            for label in soup.find_all(['label','svg']):
                label.extract()

            for lista_valores in soup.find_all('div', {'class': 'dR8kXc'}):
                lista_valores.unwrap()

            # seleciona todas as tags <div> e extrai o texto
            lista_valores = [lista_valores.text.replace('\n',' ').strip() for lista_valores in soup.find_all('div')]
            detalhes_produto = {chave: valor for chave, valor in zip(lista_chaves, lista_valores)}    
            detalhes_produto = converter_dict(detalhes_produto)

        except:
            detalhes_produto = '-'

        try:
            descricao = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[2]/div[1]/div[1]/div[2]/div[2]/div').text
        except:
            descricao = '-'

        try:
            quantidade = detalhes_produto['Estoque']

        except:
            quantidade = '0'

        try:
            modelo = detalhes_produto['Modelo']

        except:
            modelo = '-'

        try:
            marca = detalhes_produto['Marca']

        except:
            marca = '-'


        homologado = ''


        
        produto = {
            'plataforma'        : 'Shopee',
            'produto_pesquisado': 'transmissor fm',
            'titulo_produto'    : titulo_produto,
            'detalhes_produto'  : detalhes_produto,
            'marca'             : marca,
            'modelo'            : modelo,
            'id_vendedor'       : id_vendedor,
            'localizacao'       : localizacao,
            'preco_minimo'      : preco_minimo,
            'preco_maximo'      : preco_maximo,
            'preco_promo_minimo': preco_promo_minimo,
            'preco_promo_maximo': preco_promo_maximo,
            'quantidade'        : quantidade,
            'descricao'         : descricao,
            'homologado'        : '',
            'vendas_anteriores' : qtde_vendida,
            'url'               : link                                            
        }        

        produtos.append(produto)        
        
    except:
        falhas.append(link)


390
380
370
360
350
340
330
320
310
300
290
280
270
260
250
240
230
220
210
200
190
180
170
160
150
140
130
120
110
100
90
80
70
60
50
40
30
20
10
0


In [86]:
len(produtos)

395

In [87]:
len(falhas)

0

In [88]:
df = pd.DataFrame(produtos)
df.to_csv(f'resultados_shopee_transmissor_fm.csv', encoding='utf-8', index = False, sep =';')

In [ ]:
# # create object for chrome options
# chrome_options = webdriver.ChromeOptions()
# # chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox') 
# chrome_options.add_argument('disable-notifications') 
# chrome_options.add_argument('--disable-infobars') 
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument("--disable-popup-blocking")
# chrome_options.add_argument("--disable-translate")
# chrome_options.add_argument("--disable-web-security")
# chrome_options.add_argument("--disable-logging")
# chrome_options.add_argument("--disable-plugins-discovery")

# user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"

# chrome_options.add_argument(f"user-agent={user_agent}")
# chrome_options.add_argument(f"--window-size=1366,768")

# options = webdriver.ChromeOptions()



# driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()),
#                               options = chrome_options)  
# driver.implicitly_wait(5)
# actions = ActionChains(driver)

### Velho

In [ ]:
# https://medium.com/@a173823/web-scraping-on-shopee-com-6180979eff78
# Regulatron01!
from bs4 import BeautifulSoup
import requests

from urllib.parse import quote

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

from time import sleep
from collections import Counter
from tqdm import tqdm
import time
import re

import pandas as pd

# import threading

# import numpy as np
# from numpy import nan
# import re
# import concurrent.futures
# %matplotlib inline
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import seaborn as sns
# import math

In [ ]:
def converter_dict(dict_original):
    dict_convertido = {}
    for chave, valor in dict_original.items():
        if '|' in valor:
            lista_valor = valor.split('|')
            dict_convertido[chave] = lista_valor
        else:
            dict_convertido[chave] = valor
    return dict_convertido

def testar_palavras(texto_pesquisado, yes_words, no_words)-> bool:
    
    """
    Testa se as palavras de yes-words e no-words estão dentro de texto_pesquisado.

    Args:
        texto_pesquisado (str): Texto que deve vir da página web. Pode mesclar o título com a descrição do produto.
        yes_words (list): lista de palavras que fazem a função retornar True.
        no_words (list): lista de palavras que fazem a função retornar False.

    Returns:
        Bolean:
    """   
    contagem_no_words = 0
    texto_pesquisado = texto_pesquisado.lower()
    
    for palavra in yes_words:
        print('========================== yes_words ')
        if palavra in texto_pesquisado:
            return True
    
    for palavra in no_words:
        print('========================== no words ')
        if palavra in texto_pesquisado:
            contagem_no_words+= 1
    
    print ('total:', contagem_no_words)
    if contagem_no_words > 0:
        return False
    else:
        return True    

In [ ]:
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords

In [ ]:
def pesquisar_palavras_semanticas(texto, produto_pesquisado, dataframe):
    # Tokenização do texto
    tokens = nltk.word_tokenize(texto)
    
    # Definição do campo semântico
    campo_semantico = set()
    
    # Carregar stop words em português
    stop_words = set(stopwords.words('portuguese'))
    
    # Busca de sinônimos para as palavras-chave
    for token in tokens:
        synsets = wordnet.synsets(token)
        for synset in synsets:
            for lemma in synset.lemmas():
                word = lemma.name().lower()
                if word not in stop_words:
                    campo_semantico.add(word)
    
    # Verificação nos campos do dataframe
    for _, row in dataframe.iterrows():
        if row['produto_pesquisado'] == produto_pesquisado:
            if any(word.lower() in campo_semantico for word in nltk.word_tokenize(row['titulo_produto'])):
                return True
            if any(word.lower() in campo_semantico for word in nltk.word_tokenize(row['marca'])):
                return True
            if any(word.lower() in campo_semantico for word in nltk.word_tokenize(row['detalhes_produto'])):
                return True
#             if any(word.lower() in campo_semantico for word in nltk.word_tokenize(row['categoria'])):
#                 return True
            if any(word.lower() in campo_semantico for word in nltk.word_tokenize(row['descricao'])):
                return True
            if any(word.lower() in campo_semantico for word in nltk.word_tokenize(row['modelo'])):
                return True
    
    return False


In [ ]:
query = "carregador de celular"
query = quote(query)
referer_url = f'https://shopee.com.br/search?keyword={query}'

# create object for chrome options
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox') 
chrome_options.add_argument('disable-notifications') 
chrome_options.add_argument('--disable-infobars') 

referer_url = f'https://shopee.com.br/search?keyword={query}'
chrome_options.add_argument(f"Referer: {referer_url}")

driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()),
                              options = chrome_options)  
driver.implicitly_wait(5)
actions = ActionChains(driver)



In [ ]:
# Carregar a URL desejada
driver.get(referer_url)
driver.maximize_window()

# Obter os cabeçalhos da página usando o WebDriver
headers = driver.execute_script("return window.performance.getEntries()[0].responseHeaders")

# Converter os cabeçalhos para o formato correto
headers_dict = {}
for header in headers:
    name, value = header.split(':', 1)
    headers_dict[name.strip()] = value.strip()

# # Modificar os cabeçalhos conforme necessário
# headers_dict['Meu-Cabecalho'] = 'Valor do meu cabeçalho'

# Fazer uma solicitação GET com os cabeçalhos modificados usando a biblioteca requests
response = requests.get(url, headers=headers_dict)

# Imprimir a resposta
print(response.text)

# Fechar o navegador
# driver.quit()


In [ ]:
search_box = driver.find_element(By.CLASS_NAME, 'shopee-searchbar-input__input')
actions.move_to_element(search_box).click().perform()
time.sleep(0.5)

# pesquisar
limite_de_produtos = 0
set_produtos = set() 
query = 'carregador de celular'
time.sleep(0.5)

for i in query:
    search_box.send_keys(i)
    time.sleep(0.5)

header = driver.find_element(By.TAG_NAME,'header')
button = header.find_elements(By.TAG_NAME,'button')[0]
actions.move_to_element(button).click().perform()
time.sleep(3)

In [ ]:
arquivo = '20230517_Classificação_resultado_shopee_v2.csv'
try:
    df = pd.read_csv(arquivo, encoding='utf-8', sep = ';')
except UnicodeDecodeError:
    df = pd.read_csv(arquivo, encoding='latin-1', sep = ';')

df.head()

In [ ]:
df_shopee = pd.read_csv('resultado_shopee_v2.csv', sep = ';', usecols=['produto_pesquisado','url'])
df_shopee.head()

In [ ]:
print()

In [ ]:
url = df_shopee['url'][0]
driver.get(url)
sleep(5)


try:
    titulo_produto = driver.find_element(By.XPATH,
                                         '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[1]'
                                        ).text
except:
    titulo_produto = '-'


try:
    detalhes_produto = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[2]/div[1]/div[1]/div[1]/div[2]'
                                          ).get_attribute('innerHTML')
    detalhes_produto = detalhes_produto.replace('</a>','|</a>')

    lista_chaves = re.findall(r'<label[^>]*>(.*?)</label>', detalhes_produto)

    # criar um objeto Beautiful Soup a partir do HTML do div
    soup = BeautifulSoup(detalhes_produto, 'html.parser')

    # remover todas as tags <label> e svg
    for label in soup.find_all(['label','svg']):
        label.extract()

    for lista_valores in soup.find_all('div', {'class': 'dR8kXc'}):
        lista_valores.unwrap()

    # seleciona todas as tags <div> e extrai o texto
    lista_valores = [lista_valores.text.replace('\n',' ').strip() for lista_valores in soup.find_all('div')]
    detalhes_produto = {chave: valor for chave, valor in zip(lista_chaves, lista_valores)}    
    detalhes_produto = converter_dict(detalhes_produto)

except:
    detalhes_produto = '-'

try:
    descricao = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[2]/div[1]/div[1]/div[2]/div[2]/div').text
except:
    descricao = '-'


try:
    modelo = detalhes_produto['Modelo']

except:
    modelo = '-'

try:
    marca = detalhes_produto['Marca']

except:
    marca = '-'


texto              = f'{titulo_produto} {marca} {detalhes_produto} {descricao} {modelo}'
produto_pesquisado = df_shopee['produto_pesquisado'][0]

pesquisar_palavras_semanticas(texto, produto_pesquisado, df)

In [ ]:
# create object for chrome options
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox') 
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()),
                              options = chrome_options)  
driver.implicitly_wait(5)
actions = ActionChains(driver)

# Maximiza a janela
driver.maximize_window()
# navegar até a shopee
driver.get('https://shopee.com.br/')
time.sleep(3)
elements = driver.find_elements(By.TAG_NAME, 'button')

# clica em aceitar todos os cookies
for e in elements:
    if e.text == 'Aceitar todos os cookies':
        actions.move_to_element(e)
        actions.perform()
        e.click()
        break
        
time.sleep(3)

# clica em algum lugar da página
element = driver.find_element(By.ID, 'main')
actions.move_to_element(element).click().perform()
time.sleep(3)

# clicar para entrar
e = driver.find_elements(By.PARTIAL_LINK_TEXT, 'Entre')[0]
actions.move_to_element(e).click().perform()

# fazer login
telefone = ''
senha    = ''

campo_telefone = driver.find_elements(By.CLASS_NAME, 'pDzPRp')[0]
campo_senha = driver.find_elements(By.CLASS_NAME, 'pDzPRp')[1]

actions.move_to_element(campo_telefone).click().perform()
for i in telefone:
    campo_telefone.send_keys(i)
    time.sleep(0.5)

actions.move_to_element(campo_senha).click().perform()
for i in senha:
    campo_senha.send_keys(i)
    time.sleep(0.5)

# clicar para entrar
elements = driver.find_elements(By.TAG_NAME, 'button')
for e in elements:
    if e.text == 'ENTRE':
        actions.move_to_element(e).click().perform()
        break
        
time.sleep(3)

# clica em algum lugar da página
e = driver.find_element(By.CSS_SELECTOR, 'body')
actions.move_to_element(e).click().perform()
time.sleep(3)
search_box = driver.find_element(By.CLASS_NAME, 'shopee-searchbar-input__input')
actions.move_to_element(search_box).click().perform()
time.sleep(0.5)

# pesquisar
limite_de_produtos = 0
set_produtos = set() 
query = 'carregador de celular'
time.sleep(0.5)

for i in query:
    search_box.send_keys(i)
    time.sleep(0.5)

header = driver.find_element(By.TAG_NAME,'header')
button = header.find_elements(By.TAG_NAME,'button')[0]
actions.move_to_element(button).click().perform()
time.sleep(3)



## percorrer páginas
try:
    ultima_pagina = int(driver.find_element(By.XPATH,'//*[@id="main"]/div/div[2]/div/div/div[2]/div[3]/div[1]/div[2]/div/span[2]').text)
except Exception as e:
    print(e)
    try:
        # <span class="shopee-mini-page-controller__total">6</span>
        ultima_pagina = int(driver.find_elements(By.CLASS_NAME,'shopee-mini-page-controller__total')[0].text)
    except Exception as e:
        print(e)
        ultima_pagina = 1

for i in range(ultima_pagina):
    if ( len(set_produtos) == limite_de_produtos ) and ( limite_de_produtos > 0 ) :
        break
    else:
        print('pesquisando pág', i+1, 'de', ultima_pagina)

        # rolagem para esperar carregar os produtos
        for i in range(50):
            try:
                driver.execute_script("window.scrollBy(0, 100)")
                time.sleep(0.5)
            except:
                break


        time.sleep(3)
        div_links = driver.find_element(By.CSS_SELECTOR, '#main > div > div.dYFPlI > div > div > div.sdzgsX > div.shopee-search-item-result > div.row.shopee-search-item-result__items')
        a_elements = div_links.find_elements(By.TAG_NAME, 'a')

        for a in a_elements:
            set_produtos.add(a.get_attribute('href'))

        button = driver.find_elements(By.CLASS_NAME, 'shopee-icon-button--right')[0]
        actions.move_to_element(button).click().perform()
        time.sleep(3)

        
dict_produtos = { # vai guardar todas as informações dos produtos
    'produto_pesquisado' : [],
    'titulo_produto' : [],
    'id_vendedor': [],
    'quantidade' : [],
    'localizacao' : [],
    'marca' : [],
    'preco_minimo': [],
    'preco_maximo': [],
    'preco_promo_minimo': [],
    'preco_promo_maximo': [],
    'detalhes_produto' : [],
    'quantidade': [],
    'descricao': [] ,
    'homologado': [],
    'plataforma' : [],
    'url' : [],
    'modelo' : []
}

faltam = len(set_produtos)

print('pesquisando produtos')

for link in set_produtos:
    print('faltam', faltam, 'produtos')
    faltam = faltam -1
    driver.get(link)
    
    time.sleep(3)

    for i in range(10):
        try:
            driver.execute_script("window.scrollBy(0, 100)")
            time.sleep(0.5)
        except:
            break

    try:
        titulo_produto = driver.find_element(By.XPATH,
                                             '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[1]'
                                            ).text
    except:
        titulo_produto = '-'

    try:
        id_vendedor    = driver.find_element(By.XPATH,
                                             '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[1]/div[1]/div/div[1]'
                                            ).text
    except:
        id_vendedor    = '-'

    try:      
        qtde_vendida = driver.find_element(By.XPATH,
                                           '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[2]/div[2]/div[1]'
                                          ).text
    except:
        qtde_vendida = '0'

    try:
        localizacao = driver.find_element(By.XPATH,
                                            '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[4]/div/div[2]/div/div[2]/div[2]/div[1]/div[2]/div[1]'
                                         ).text
    except:
        localizacao = ''

    try:
        preco = driver.find_element(By.XPATH,
                                   '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[3]/div/div/div/div/div[1]'
                                   ).text

        if preco.find('-'):
            preco = preco.split('-')
            preco_minimo = preco[0][2:]
            preco_maximo = preco[-1][3:]
        else:
            preco_minimo = preco[2:]
            preco_maximo = '-'
    except:
        preco_minimo = '-'
        preco_maximo = '-'

    try:
        preco_promo = driver.find_element(By.XPATH,
                                   '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[3]/div/div/div/div/div[2]/div[1]'
                                   ).text

        if preco_promo.find('-'):
            preco_promo = preco_promo.split('-')
            preco_promo_minimo = preco_promo[0][2:]
            preco_promo_maximo = preco_promo[-1][3:]
        else:
            preco_promo_minimo = preco_promo[2:]
            preco_promo_maximo = '-'

    except:
            preco_promo_minimo = '-'
            preco_promo_maximo = '-'


    try:
        detalhes_produto = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[2]/div[1]/div[1]/div[1]/div[2]'
                                              ).get_attribute('innerHTML')
        detalhes_produto = detalhes_produto.replace('</a>','|</a>')

        lista_chaves = re.findall(r'<label[^>]*>(.*?)</label>', detalhes_produto)

        # criar um objeto Beautiful Soup a partir do HTML do div
        soup = BeautifulSoup(detalhes_produto, 'html.parser')

        # remover todas as tags <label> e svg
        for label in soup.find_all(['label','svg']):
            label.extract()

        for lista_valores in soup.find_all('div', {'class': 'dR8kXc'}):
            lista_valores.unwrap()

        # seleciona todas as tags <div> e extrai o texto
        lista_valores = [lista_valores.text.replace('\n',' ').strip() for lista_valores in soup.find_all('div')]
        detalhes_produto = {chave: valor for chave, valor in zip(lista_chaves, lista_valores)}    
        detalhes_produto = converter_dict(detalhes_produto)

    except:
        detalhes_produto = '-'

    try:
        descricao = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[2]/div[1]/div[1]/div[2]/div[2]/div').text
    except:
        descricao = '-'

    try:
        quantidade = detalhes_produto['Estoque']

    except:
        quantidade = '0'

    try:
        modelo = detalhes_produto['Modelo']

    except:
        modelo = '-'
        
    try:
        marca = detalhes_produto['Marca']

    except:
        marca = '-'

    # if testar_palavras(f'{titulo_produto} {descricao} {detalhes_produto.values()}', yes_words, no_words):
    #     homologado = 'CANDIDATO'
    # else:
    #     homologado = ''
    homologado = ''

    dict_produtos['produto_pesquisado'].append(query)
    dict_produtos['titulo_produto'].append(titulo_produto)
    dict_produtos['id_vendedor'].append(id_vendedor)
    dict_produtos['quantidade'].append(quantidade)
    dict_produtos['localizacao'].append(localizacao)
    dict_produtos['marca'].append(marca)
    dict_produtos['preco_minimo'].append(preco_minimo)
    dict_produtos['preco_maximo'].append(preco_maximo)
    dict_produtos['preco_promo_minimo'].append(preco_promo_minimo)
    dict_produtos['preco_promo_maximo'].append(preco_promo_maximo)
    dict_produtos['detalhes_produto'].append(detalhes_produto)
    dict_produtos['descricao'].append(descricao)
    dict_produtos['homologado'].append(homologado)
    dict_produtos['plataforma'].append('Shopee')
    dict_produtos['url'].append(link)
    dict_produtos['modelo'].append(modelo)



df = pd.DataFrame(dict_produtos)
df.to_csv(f'resultados_shopee_{query}.csv', encoding='utf-8', index = False, sep =';')

In [ ]:
df = pd.DataFrame(dict_produtos)
df.to_csv(f'resultados_shopee_{query}_01.csv', encoding='utf-8', index = False, sep =';')

In [ ]:
for link in set_produtos_aux:
    print('faltam', faltam, 'produtos')
    faltam = faltam -1
    driver.get(link)
    
    time.sleep(3)

    for i in range(10):
        try:
            driver.execute_script("window.scrollBy(0, 100)")
            time.sleep(0.5)
        except:
            break

    try:
        titulo_produto = driver.find_element(By.XPATH,
                                             '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[1]'
                                            ).text
    except:
        titulo_produto = '-'

    try:
        id_vendedor    = driver.find_element(By.XPATH,
                                             '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[1]/div[1]/div/div[1]'
                                            ).text
    except:
        id_vendedor    = '-'

    try:      
        qtde_vendida = driver.find_element(By.XPATH,
                                           '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[2]/div[2]/div[1]'
                                          ).text
    except:
        qtde_vendida = '0'

    try:
        localizacao = driver.find_element(By.XPATH,
                                            '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[4]/div/div[2]/div/div[2]/div[2]/div[1]/div[2]/div[1]'
                                         ).text
    except:
        localizacao = ''

    try:
        preco = driver.find_element(By.XPATH,
                                   '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[3]/div/div/div/div/div[1]'
                                   ).text

        if preco.find('-'):
            preco = preco.split('-')
            preco_minimo = preco[0][2:]
            preco_maximo = preco[-1][3:]
        else:
            preco_minimo = preco[2:]
            preco_maximo = '-'
    except:
        preco_minimo = '-'
        preco_maximo = '-'

    try:
        preco_promo = driver.find_element(By.XPATH,
                                   '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[3]/div/div/div/div/div[2]/div[1]'
                                   ).text

        if preco_promo.find('-'):
            preco_promo = preco_promo.split('-')
            preco_promo_minimo = preco_promo[0][2:]
            preco_promo_maximo = preco_promo[-1][3:]
        else:
            preco_promo_minimo = preco_promo[2:]
            preco_promo_maximo = '-'

    except:
            preco_promo_minimo = '-'
            preco_promo_maximo = '-'


    try:
        detalhes_produto = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[2]/div[1]/div[1]/div[1]/div[2]'
                                              ).get_attribute('innerHTML')
        detalhes_produto = detalhes_produto.replace('</a>','|</a>')

        lista_chaves = re.findall(r'<label[^>]*>(.*?)</label>', detalhes_produto)

        # criar um objeto Beautiful Soup a partir do HTML do div
        soup = BeautifulSoup(detalhes_produto, 'html.parser')

        # remover todas as tags <label> e svg
        for label in soup.find_all(['label','svg']):
            label.extract()

        for lista_valores in soup.find_all('div', {'class': 'dR8kXc'}):
            lista_valores.unwrap()

        # seleciona todas as tags <div> e extrai o texto
        lista_valores = [lista_valores.text.replace('\n',' ').strip() for lista_valores in soup.find_all('div')]
        detalhes_produto = {chave: valor for chave, valor in zip(lista_chaves, lista_valores)}    
        detalhes_produto = converter_dict(detalhes_produto)

    except:
        detalhes_produto = '-'

    try:
        descricao = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[2]/div[1]/div[1]/div[2]/div[2]/div').text
    except:
        descricao = '-'

    try:
        quantidade = detalhes_produto['Estoque']

    except:
        quantidade = '0'

    try:
        modelo = detalhes_produto['Modelo']

    except:
        modelo = '-'
        
    try:
        marca = detalhes_produto['Marca']

    except:
        marca = '-'

    # if testar_palavras(f'{titulo_produto} {descricao} {detalhes_produto.values()}', yes_words, no_words):
    #     homologado = 'CANDIDATO'
    # else:
    #     homologado = ''
    homologado = ''

    dict_produtos['produto_pesquisado'].append(query)
    dict_produtos['titulo_produto'].append(titulo_produto)
    dict_produtos['id_vendedor'].append(id_vendedor)
    dict_produtos['quantidade'].append(quantidade)
    dict_produtos['localizacao'].append(localizacao)
    dict_produtos['marca'].append(marca)
    dict_produtos['preco_minimo'].append(preco_minimo)
    dict_produtos['preco_maximo'].append(preco_maximo)
    dict_produtos['preco_promo_minimo'].append(preco_promo_minimo)
    dict_produtos['preco_promo_maximo'].append(preco_promo_maximo)
    dict_produtos['detalhes_produto'].append(detalhes_produto)
    dict_produtos['descricao'].append(descricao)
    dict_produtos['homologado'].append(homologado)
    dict_produtos['plataforma'].append('Shopee')
    dict_produtos['url'].append(link)
    dict_produtos['modelo'].append(modelo)



df = pd.DataFrame(dict_produtos)
df.to_csv(f'resultados_shopee_{query}_02.csv', encoding='utf-8', index = False, sep =';')